In [18]:
%matplotlib inline
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn
from torchsummary import summary
from d2l import torch as d2l
import numpy as np
  

In [2]:
transform = transforms.Compose(
      [transforms.ToTensor(),
      transforms.Resize(224),
       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

batch_size = 10

#validation and training set
full_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

#download 10000 testing dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#split 50k dataset into train and validation
train_size = int(0.7 * len(full_dataset))
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def initialize_model(model_name, num_classes=10, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = torchvision.models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        nn.init.xavier_uniform_(model_ft.fc.weight);

    elif model_name == "alexnet":
        """ Alexnet
        """
        print("alexnet")
        model_ft = torchvision.models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        nn.init.xavier_uniform_(model_ft.classifier[6].weight);

    elif model_name == "vgg":
        """ VGG16
        """
        print("vgg")
        model_ft = torchvision.models.vgg16(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        nn.init.xavier_uniform_(model_ft.classifier[6].weight);
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

### softmax regression on original input

In [ ]:
# PyTorch does not implicitly reshape the inputs. Thus we define the flatten
# layer to reshape the inputs before the linear layer in our network
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)
    
val_iter = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

net = nn.Sequential(nn.Flatten(), nn.Linear(150528, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.1)
num_epochs = 5
#replace with ch13 for parallel
d2l.train_ch13(net, train_iter, val_iter, loss, trainer, num_epochs,
                   d2l.try_all_gpus())

### softmax on last convolution layer

In [35]:
#softmax 
class SoftmaxRegression(nn.Module):
  def __init__(self, input_dim, output_dim, *args, **kwargs):
    super(SoftmaxRegression, self).__init__()
    self.layer = nn.Linear(input_dim, output_dim)

  def forward(self, X, *args, **kwargs):
    return F.softmax(self.layer(X), dim=-1)

softmax = SoftmaxRegression(9216, 10)

class NewModel(nn.Module):
  def __init__(self, pretrained_model, output_model, num_layers):
    super(NewModel, self).__init__()
    self.pretrained = pretrained_model
    self.flatten = nn.Flatten()
    self.output = output_model
    #https://forums.fast.ai/t/pytorch-best-way-to-get-at-intermediate-layers-in-vgg-and-resnet/5707
    self.features = nn.Sequential(*list(self.pretrained.children())[:num_layers])
  
  def forward(self, x):
    features = self.flatten.forward(self.features(x))
    return self.output.forward(features)

In [36]:
resnet = initialize_model('resnet')
#print(resnet)
net = NewModel(resnet, softmax, 8)
d2l.train_ch13(net, train_iter, val_iter, loss, trainer, num_epochs,
                   d2l.try_all_gpus())

### softmax on an intermediate layer

In [ ]:
resnet = initialize_model('resnet')
#print(resnet)
net = NewModel(resnet, softmax, 4)
d2l.train_ch13(net, train_iter, val_iter, loss, trainer, num_epochs,
                   d2l.try_all_gpus())